<a href="https://colab.research.google.com/github/carolsworld/ICS-test-bed-PoC/blob/anomalydetection/TRIST_AnomalyDetection(with_0_10_fraction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Demonstration of using datasets generated from TRIST for developing Anomaly Detection Model**

##Getting Started
Anomaly detection is a common technique used for identifying abnormal or rare observations that significantly different from the majority of the data in a dataset. This technique is applicable to detection of anomalous events happened on Internet of Things (IoT) and many other real-life problems.

The purpose of this demonstration is to **validate whether an anomaly detection model trained based on datasets of normal operations could effectively identify anomalies on datasets generated during attacks**.

##Types of Anomaly Detection Models
 **Unsupervised model** is used in this demonstration, meaning that we do not have any predefined labels on the dataset for model training.

The dataset generated from TRIST representing the **normal** operations of the water supply and storage is used for model training. In other words, the model is assuming that the majority of the instances are normal. This assumption is reasonable as we are generating data from python script without interference from physical environment.

The datasets generated from TRIST that representing the attack simulations are used for prediction on unseen data. In this way, we could achieve our purpose by validating whether the trained model could effectively identify anomalies on datasets generated during attacks. If it is effective, the model could be deployed to the production environment.

After data preparation, the following PyCaret workflow is used in our machine learning model development:
* **1. Setup**
* **2. Create Model**
* **3. Assign Labels**
* **4. Analyse Model**
* **5. Save Trained Model**
* **6. Load Trained Model**
* **7. Predict Unseen Data**

There are other types of anomaly detection model, one is "supervised", another one is "semi-supervised".
* *Supervised:* Supervised model uses dataset that specifies which data records are normal, and which data records are anomaly. Thus, it requires collection of sufficient abnormal data to train a supervised anomaly detector.

* *Semi-supervised:* Semi-supervised model uses only normal data during the training process. It predicts whether new data point is normal or abnormal based on the distribution of the data in the trained model.

These aforesaid two types of model fall out of the scope of our demonstration.

## Anomaly Detection Algorithm
Data scientists have developed many anomaly detection algorithm over the past decades. For simplicity, we have used PyCaret, an open-source low-code machine learning library, to perform end-to-end machine learning model management.

In [1]:
# Install the pyCaret library and check the version
!pip install -q pycaret && pip install -q package-name
from pycaret.utils import version
print(version())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.8 MB/s eta 

#Step 1: Data Preparation

In [3]:
# Upload a file from local machine to the Google Colab environment
from google.colab import files
import io
import pandas as pd

uploaded = files.upload()

# Assuming you know the filename or if there's only one file uploaded
filename = next(iter(uploaded))
data = pd.read_csv(io.BytesIO(uploaded[filename]), parse_dates=['time'])
data['time'] = data['time'].dt.strftime('%Y-%m-%d %H:%M:%S')


Saving normal_1hr.csv to normal_1hr.csv


In [4]:
# Check the number of rows and columns in the dataframe
print("The number of rows and columns: ", data.shape)

The number of rows and columns:  (3595, 7)


In [5]:
# Print the first 5 rows of the dataframe
print("The first 5 rows of the dataset: \n", data.head())

The first 5 rows of the dataset: 
                   time  FIT101  MV101    LIT101  P101  FIT201    LIT301
0  2023-07-17 11:00:00    2.55      1  0.597593     0     0.0  0.951852
1  2023-07-17 11:00:01    2.55      1  0.644815     0     0.0  0.924074
2  2023-07-17 11:00:02    2.55      1  0.692037     0     0.0  0.896296
3  2023-07-17 11:00:03    2.55      1  0.739259     0     0.0  0.868519
4  2023-07-17 11:00:04    2.55      1  0.786481     0     0.0  0.840741


In [6]:
# Check the time span of the data
data['time'] = pd.to_datetime(data['time'])
time_span = data.time.max() - data.time.min()
print("Time span of the dataset:", time_span)

Time span of the dataset: 0 days 00:59:59


In [7]:
# Plot a graph to see how data looks like
import plotly.express as px
fig = px.line(data, x="time", y=['FIT101', 'MV101', 'LIT101', 'P101', 'FIT201', 'LIT301'], title='TRIST normal dataset', template = 'plotly_dark')
fig.show()

Explanation for the above graph:
* FIT101 = Sensor 101 measuring water flow rate
* MV101 = Motorised valve 101
* LIT101 = Sensor 101 measuring water level at Tank 101
* P101 = Pump 101
* FIT201 = Sensor 201 measuring water flow rate
* LIT301 = Sensor 301 measuring water level at Tank 301

Please refer to [SWat Testbed description](https://minicps.readthedocs.io/en/latest/swat-tutorial.html?highlight=plc1#supply-and-storage-control) for details of Supply and Storage control in water treatment plant.

Plotting all the records do not give us a clear picture of what is the data pattern of the simulation. Thus, the time feature is extracted for more accurate analysis.

In [8]:
# Algorithms cannot directly consume date or timestamp data, thus we will extract the features from the timestamp
# and will drop the actual timestamp column before training models.
# Set the timestamp column as the index of the dataframe
data.set_index('time', drop=True, inplace=True)

In [9]:
# Extract features from timestamp
data['hours'] = [i.hour for i in data.index]
data['minutes'] = [i.minute for i in data.index]
data['seconds']= [i.second for i in data.index]

print("\nThe first 5 rows of the dataset: \n", data.head())

print("\nThe last 5 rows of the dataset: \n", data.tail())


The first 5 rows of the dataset: 
                      FIT101  MV101    LIT101  P101  FIT201    LIT301  hours  \
time                                                                          
2023-07-17 11:00:00    2.55      1  0.597593     0     0.0  0.951852     11   
2023-07-17 11:00:01    2.55      1  0.644815     0     0.0  0.924074     11   
2023-07-17 11:00:02    2.55      1  0.692037     0     0.0  0.896296     11   
2023-07-17 11:00:03    2.55      1  0.739259     0     0.0  0.868519     11   
2023-07-17 11:00:04    2.55      1  0.786481     0     0.0  0.840741     11   

                     minutes  seconds  
time                                   
2023-07-17 11:00:00        0        0  
2023-07-17 11:00:01        0        1  
2023-07-17 11:00:02        0        2  
2023-07-17 11:00:03        0        3  
2023-07-17 11:00:04        0        4  

The last 5 rows of the dataset: 
                      FIT101  MV101    LIT101  P101  FIT201    LIT301  hours  \
time            

Simply listing the data record could not provide any insight. The following plots are used to analyse the data pattern of the TRIST dataset.

In [10]:
# Plot a graph to see how the first 10 minutes of data looks like
first_10_minutes = data.first('10T') # 'T' stands for minutes
fig = px.line(first_10_minutes, y=['FIT101', 'MV101', 'LIT101', 'P101', 'FIT201', 'LIT301'], title='TRIST data', template = 'plotly_dark')
fig.show()

Explanation for the above and the following graph:
* FIT101 = Sensor 101 measuring water flow rate
* MV101 = Motorised valve 101
* LIT101 = Sensor 101 measuring water level at Tank 101
* P101 = Pump 101
* FIT201 = Sensor 201 measuring water flow rate
* LIT301 = Sensor 301 measuring water level at Tank 301

Please refer to [SWat Testbed description](https://minicps.readthedocs.io/en/latest/swat-tutorial.html?highlight=plc1#supply-and-storage-control) for details of Supply and Storage control in water treatment plant.

In [11]:
# Plot a graph to see how the first 1 minutes of data looks like
first_1_minutes = data.first('1T') # 'T' stands for minutes
fig = px.line(first_1_minutes, y=['FIT101', 'MV101', 'LIT101', 'P101', 'FIT201', 'LIT301'], title='TRIST data - first 1 minute of normal operation', template = 'plotly_dark')
fig.show()

# Step 2: PyCaret Workflow

##**1. Setup**

In [12]:
# Setup PyCaret
# Session ID is added to provide a reproducible results.
from pycaret.anomaly import *
s = setup(data, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(3595, 9)"
2,Transformed data shape,"(3595, 9)"
3,Numeric features,9
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [13]:
# Find out the models available in the anomaly detection module of PyCaret
models()

,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pycaret.internal.patches.pyod.CBLOFForceToDouble
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


##**2. Create Model**

In [14]:
# Train model with Isolation Forest as an example for demonstration
# Fraction = 0.1 (i.e. 10% of the population, 10 out of 100 records will be regarded as anomaly)
# When the value of fraction increases, the number of anomaly records will be increased in proportion.
# Therefore, this value shall be fine-tuned by the specific use case and the level of risk acceptance.
iforest = create_model('iforest', fraction = 0.1)

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
# Find out more about the configurations of iforest
# Containmination is same as the 'fraction' defined in the previous code
print(iforest)

IForest(behaviour='new', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=-1, random_state=123,
    verbose=0)


##**3. Assign Labels**

In [16]:
# Two new columns are appended in the table:
# Column "Anomaly" with value 1 refers to anomaly, value 0 refers to normal
# Column "Anomaly_Score" provides the score calculated by the algorithm.
iforest_results = assign_model(iforest)
iforest_results.head()

,FIT101,MV101,LIT101,P101,FIT201,LIT301,hours,minutes,seconds,Anomaly,Anomaly_Score
time,,,,,,,,,,,
2023-07-17 11:00:00,2.55,1,0.597593,0,0.0,0.951852,11,0,0,1,0.014711
2023-07-17 11:00:01,2.55,1,0.644815,0,0.0,0.924074,11,0,1,1,0.003954
2023-07-17 11:00:02,2.55,1,0.692037,0,0.0,0.896296,11,0,2,1,0.003242
2023-07-17 11:00:03,2.55,1,0.739259,0,0.0,0.868519,11,0,3,1,0.011586
2023-07-17 11:00:04,2.55,1,0.786481,0,0.0,0.840741,11,0,4,1,0.024779


In [17]:
# Print the anomalies
print("Anomalies: \n", iforest_results[iforest_results['Anomaly'] == 1])

Anomalies: 
                      FIT101  MV101    LIT101  P101  FIT201    LIT301  hours  \
time                                                                          
2023-07-17 11:00:00    2.55      1  0.597593     0    0.00  0.951852     11   
2023-07-17 11:00:01    2.55      1  0.644815     0    0.00  0.924074     11   
2023-07-17 11:00:02    2.55      1  0.692037     0    0.00  0.896296     11   
2023-07-17 11:00:03    2.55      1  0.739259     0    0.00  0.868519     11   
2023-07-17 11:00:04    2.55      1  0.786481     0    0.00  0.840741     11   
...                     ...    ...       ...   ...     ...       ...    ...   
2023-07-17 11:59:41    2.55      1  0.498333     0    0.00  1.011667     11   
2023-07-17 11:59:47    2.55      1  0.772222     0    0.00  0.845000     11   
2023-07-17 11:59:48    2.55      0  0.819444     0    0.00  0.822778     11   
2023-07-17 11:59:49    0.00      0  0.819444     0    0.00  0.795000     11   
2023-07-17 11:59:50    0.00      0  0.7

In [18]:
# Sort records by anomaly score
sorted_results = iforest_results.sort_values(by='Anomaly_Score', ascending=False)

# Display the sorted anomalies results
print("Anomalies: \n", sorted_results[sorted_results['Anomaly'] == 1])

Anomalies: 
                      FIT101  MV101    LIT101  P101  FIT201    LIT301  hours  \
time                                                                          
2023-07-17 11:55:02    0.00      0  0.803889     1     0.0  0.778148     11   
2023-07-17 11:02:00    0.00      0  0.827778     0     0.0  0.785000     11   
2023-07-17 11:59:04    0.00      0  0.826481     0     0.0  0.795741     11   
2023-07-17 11:00:06    0.00      0  0.824259     0     0.0  0.785185     11   
2023-07-17 11:00:52    0.00      0  0.808889     0     0.0  0.778333     11   
...                     ...    ...       ...   ...     ...       ...    ...   
2023-07-17 11:29:38    0.00      0  0.804259     0     0.0  0.833333     11   
2023-07-17 11:45:14    0.00      0  0.817222     0     0.0  0.802037     11   
2023-07-17 11:05:34    2.55      1  0.497222     0     0.0  1.001111     11   
2023-07-17 11:41:54    2.55      1  0.815556     0     0.0  0.824630     11   
2023-07-17 11:29:17    0.00      0  0.8

The highest anomaly score is 0.076201 and the lowest anomaly score is 0.000020.

In [19]:
# Count the number of anomalies
number_of_anomalies = iforest_results['Anomaly'].sum()

print("Number of anomalies detected:", number_of_anomalies)

Number of anomalies detected: 360


In [20]:
# Recall the number of rows and columns in the dataset
print("The number of rows and columns: ", data.shape)

The number of rows and columns:  (3595, 9)


The number of anomalies detected is 360, representing 10% of the 3595 records in the datasets.

This 10% is the fraction parameter we have defined at 'Step 2.2. Create Model' of the PyCaret Workflow.

The number of anomalies assigned by the model will increase in proportion of the percentage of fraction parameter we define.  

##**4. Analyse Model**

In [21]:
# Plot anomalies
import plotly.graph_objects as go

# Plot value on y-axis and date on x-axis
fig = px.line(iforest_results, x=iforest_results.index, y=['FIT101', 'MV101', 'LIT101', 'P101', 'FIT201', 'LIT301'], title='TRIST Anomaly Detection - Plot all the anomalies identified', template = 'plotly_dark')

# Create list of outliers
outliers = iforest_results[iforest_results['Anomaly'] == 1].index

# obtain y value of anomalies to plot
y_values1 = [iforest_results.loc[i]['FIT101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values1, mode = 'markers',
                name = 'FIT101 Anomalies',
                marker=dict(color='yellow',size=10)))

# obtain y value of anomalies to plot
y_values2 = [iforest_results.loc[i]['MV101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values2, mode = 'markers',
                name = 'MV101 Anomalies',
                marker=dict(color='white',size=10)))

# obtain y value of anomalies to plot
y_values3 = [iforest_results.loc[i]['LIT101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values3, mode = 'markers',
                name = 'LIT101 Anomalies',
                marker=dict(color='pink',size=10)))

# obtain y value of anomalies to plot
y_values4 = [iforest_results.loc[i]['P101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values4, mode = 'markers',
                name = 'P101 Anomalies',
                marker=dict(color='lightcyan',size=10)))

# obtain y value of anomalies to plot
y_values5 = [iforest_results.loc[i]['FIT201'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values5, mode = 'markers',
                name = 'FIT201 Anomalies',
                marker=dict(color='lightgoldenrodyellow',size=10)))

# obtain y value of anomalies to plot
y_values6 = [iforest_results.loc[i]['LIT301'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values6, mode = 'markers',
                name = 'LIT301 Anomalies',
                marker=dict(color='lightyellow',size=10)))

fig.show()

Explanation for the above graph:
* FIT101 = Sensor 101 measuring water flow rate
* MV101 = Motorised valve 101
* LIT101 = Sensor 101 measuring water level at Tank 101
* P101 = Pump 101
* FIT201 = Sensor 201 measuring water flow rate
* LIT301 = Sensor 301 measuring water level at Tank 301

Please refer to [SWat Testbed description](https://minicps.readthedocs.io/en/latest/swat-tutorial.html?highlight=plc1#supply-and-storage-control) for details of Supply and Storage control in water treatment plant.

In [22]:
# Plot anomalies in 3D with t-SNE
plot_model(iforest, plot = 'tsne')

##**5. Save Trained Model**

In [23]:
# Save pipeline into pickle file
# Download the pickle file from Google Colab's "Files" section on the left-hand side bar if you want to use the model later.
save_model(iforest, 'iforest_0.10_anomalous')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['FIT101', 'MV101', 'LIT101',
                                              'P101', 'FIT201', 'LIT301',
                                              'hours', 'minutes', 'seconds'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('trained_model',
                  IForest(behaviour='new', bootstrap=False, contamination=0.1, max_features=1.0,
     max_samples='auto', n_estimators=100, n_jobs=-1, random_state=123,
     verbose=0))]),
 'iforest_0.10_anomalous.pkl')

##**6. Load Trained Model**

In [24]:
# Load the trained model (i.e. the pickle file created in previous step)
# Or upload the pickle file (iforest_X_anomalous.pkl) from your local machine if you have stopped the training process previously.
loaded_iforest_pipeline = load_model('iforest_0.10_anomalous')
loaded_iforest_pipeline

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['FIT101', 'MV101', 'LIT101',
                                             'P101', 'FIT201', 'LIT301',
                                             'hours', 'minutes', 'seconds'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('trained_model',
                 IForest(behaviour='new', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=-1, random_state=123,
    verbose=0))])

##**7. Predict Unseen Data**

### A. Unseen Attack 1

* Method: Keep MV101 open until tank T101 overflow

* Result: tank T101 overflow

#### 1.Data Preparation

In [25]:
# Upload a file from local machine to the Google Colab environment
from google.colab import files
import io
import pandas as pd

uploaded = files.upload()

# Assuming you know the filename or if there's only one file uploaded
filename = next(iter(uploaded))
unseen_data1 = pd.read_csv(io.BytesIO(uploaded[filename]), parse_dates=['time'])
unseen_data1['time'] = unseen_data1['time'].dt.strftime('%Y-%m-%d %H:%M:%S')


Saving attack1.csv to attack1.csv


In [26]:
# Check the number of rows and columns in the dataframe
print("The number of rows and columns: ", unseen_data1.shape)

The number of rows and columns:  (75, 7)


In [27]:
# Print the first 5 rows of the dataframe
print("The first 5 rows of the dataset: \n", unseen_data1.head())

The first 5 rows of the dataset: 
                   time  FIT101  MV101    LIT101  P101  FIT201    LIT301
0  2023-07-31 16:45:06    2.55      1  0.500741     1    2.45  0.507037
1  2023-07-31 16:45:07    2.55      1  0.502593     1    2.45  0.524630
2  2023-07-31 16:45:08    2.55      1  0.504444     1    2.45  0.542222
3  2023-07-31 16:45:09    2.55      1  0.505926     1    2.45  0.556296
4  2023-07-31 16:45:10    2.55      1  0.507778     1    2.45  0.573889


In [28]:
# Check the time span of the unseen data
unseen_data1['time'] = pd.to_datetime(unseen_data1['time'])
time_span = unseen_data1.time.max() - unseen_data1.time.min()
print("Time span of the dataset:", time_span)

Time span of the dataset: 0 days 00:01:14


In [29]:
# Plot a graph to see how data looks like
import plotly.express as px
fig = px.line(unseen_data1, x="time", y=['FIT101', 'MV101', 'LIT101', 'P101', 'FIT201', 'LIT301'], title='TRIST unseen data - attack 1 (Keep MV101 open until tank T101 overflow)', template = 'plotly_dark')
fig.show()

In [30]:
# Algorithms cannot directly consume date or timestamp data, thus we will extract the features from the timestamp
# and will drop the actual timestamp column before training models.
# Set the timestamp column as the index of the dataframe
unseen_data1.set_index('time', drop=True, inplace=True)

In [31]:
# Extract features from timestamp
unseen_data1['hours'] = [i.hour for i in unseen_data1.index]
unseen_data1['minutes'] = [i.minute for i in unseen_data1.index]
unseen_data1['seconds']= [i.second for i in unseen_data1.index]

print("\nThe first 5 rows of the dataset: \n", unseen_data1.head())

print("\nThe last 5 rows of the dataset: \n", unseen_data1.tail())


The first 5 rows of the dataset: 
                      FIT101  MV101    LIT101  P101  FIT201    LIT301  hours  \
time                                                                          
2023-07-31 16:45:06    2.55      1  0.500741     1    2.45  0.507037     16   
2023-07-31 16:45:07    2.55      1  0.502593     1    2.45  0.524630     16   
2023-07-31 16:45:08    2.55      1  0.504444     1    2.45  0.542222     16   
2023-07-31 16:45:09    2.55      1  0.505926     1    2.45  0.556296     16   
2023-07-31 16:45:10    2.55      1  0.507778     1    2.45  0.573889     16   

                     minutes  seconds  
time                                   
2023-07-31 16:45:06       45        6  
2023-07-31 16:45:07       45        7  
2023-07-31 16:45:08       45        8  
2023-07-31 16:45:09       45        9  
2023-07-31 16:45:10       45       10  

The last 5 rows of the dataset: 
                      FIT101  MV101    LIT101  P101  FIT201    LIT301  hours  \
time            

#### 2.Prediction

In [32]:
# Predict on test set
# The predict_model function returns Anomaly and Anomaly_Score label as a new column in the input dataframe.
# The predict_model is only useful when you want to obtain labels on unseen data (i.e. data that was not used during training the model).
iforest_pred1 = predict_model(iforest, data=unseen_data1)
iforest_pred1

,FIT101,MV101,LIT101,P101,FIT201,LIT301,hours,minutes,seconds,Anomaly,Anomaly_Score
time,,,,,,,,,,,
2023-07-31 16:45:06,2.55,1.0,0.500741,1.0,2.45,0.507037,16.0,45.0,6.0,1,0.005184
2023-07-31 16:45:07,2.55,1.0,0.502593,1.0,2.45,0.524630,16.0,45.0,7.0,1,0.004393
2023-07-31 16:45:08,2.55,1.0,0.504444,1.0,2.45,0.542222,16.0,45.0,8.0,1,0.000275
2023-07-31 16:45:09,2.55,1.0,0.505926,1.0,2.45,0.556296,16.0,45.0,9.0,1,0.004249
2023-07-31 16:45:10,2.55,1.0,0.507778,1.0,2.45,0.573889,16.0,45.0,10.0,1,0.008487
...,...,...,...,...,...,...,...,...,...,...,...
2023-07-31 16:46:16,2.55,1.0,1.202222,1.0,0.00,0.536296,16.0,46.0,16.0,1,0.032480
2023-07-31 16:46:17,2.55,1.0,1.202222,1.0,0.00,0.508519,16.0,46.0,17.0,1,0.032205
2023-07-31 16:46:18,2.55,1.0,1.202222,1.0,0.00,0.480741,16.0,46.0,18.0,1,0.032121


In [33]:
# Print the predicted anomalies
print("Anomalies: \n", iforest_pred1[iforest_pred1['Anomaly'] == 1])

Anomalies: 
                      FIT101  MV101    LIT101  P101  FIT201    LIT301  hours  \
time                                                                          
2023-07-31 16:45:06    2.55    1.0  0.500741   1.0    2.45  0.507037   16.0   
2023-07-31 16:45:07    2.55    1.0  0.502593   1.0    2.45  0.524630   16.0   
2023-07-31 16:45:08    2.55    1.0  0.504444   1.0    2.45  0.542222   16.0   
2023-07-31 16:45:09    2.55    1.0  0.505926   1.0    2.45  0.556296   16.0   
2023-07-31 16:45:10    2.55    1.0  0.507778   1.0    2.45  0.573889   16.0   
2023-07-31 16:45:11    2.55    1.0  0.509630   1.0    2.45  0.591481   16.0   
2023-07-31 16:45:12    2.55    1.0  0.511481   1.0    2.45  0.609074   16.0   
2023-07-31 16:45:13    2.55    1.0  0.513333   1.0    2.45  0.626667   16.0   
2023-07-31 16:45:14    2.55    1.0  0.515185   1.0    2.45  0.644259   16.0   
2023-07-31 16:45:15    2.55    1.0  0.517037   1.0    2.45  0.661852   16.0   
2023-07-31 16:45:16    2.55    1.0  0.5

In [ ]:
# Count the number of anomalies
number_of_anomalies = iforest_pred1['Anomaly'].sum()

print("Number of anomalies detected in attack 1:", number_of_anomalies)

Number of anomalies detected in attack 1: 45


In [34]:
# Recall the number of rows and columns in the dataset
print("The number of rows and columns: ", unseen_data1.shape)

The number of rows and columns:  (75, 9)


In [35]:
# Plot anomalies
import plotly.graph_objects as go

# Plot value on y-axis and date on x-axis
fig = px.line(iforest_pred1, x=iforest_pred1.index, y=['FIT101', 'MV101', 'LIT101', 'P101', 'FIT201', 'LIT301'], title='TRIST Anomaly Detection - Prediction on Unseen Attack 1 (Keep MV101 open until tank T101 overflow)', template = 'plotly_dark')

# Create list of outliers
outliers = iforest_pred1[iforest_pred1['Anomaly'] == 1].index

# obtain y value of anomalies to plot
y_values1 = [iforest_pred1.loc[i]['FIT101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values1, mode = 'markers',
                name = 'FIT101 Anomalies',
                marker=dict(color='yellow',size=10)))

# obtain y value of anomalies to plot
y_values2 = [iforest_pred1.loc[i]['MV101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values2, mode = 'markers',
                name = 'MV101 Anomalies',
                marker=dict(color='white',size=10)))

# obtain y value of anomalies to plot
y_values3 = [iforest_pred1.loc[i]['LIT101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values3, mode = 'markers',
                name = 'LIT101 Anomalies',
                marker=dict(color='pink',size=10)))

# obtain y value of anomalies to plot
y_values4 = [iforest_pred1.loc[i]['P101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values4, mode = 'markers',
                name = 'P101 Anomalies',
                marker=dict(color='lightcyan',size=10)))

# obtain y value of anomalies to plot
y_values5 = [iforest_pred1.loc[i]['FIT201'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values5, mode = 'markers',
                name = 'FIT201 Anomalies',
                marker=dict(color='lightgoldenrodyellow',size=10)))

# obtain y value of anomalies to plot
y_values6 = [iforest_pred1.loc[i]['LIT301'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values6, mode = 'markers',
                name = 'LIT301 Anomalies',
                marker=dict(color='lightyellow',size=10)))

fig.show()

Explanation for the above graph:
* FIT101 = Sensor 101 measuring water flow rate
* MV101 = Motorised valve 101
* LIT101 = Sensor 101 measuring water level at Tank 101
* P101 = Pump 101
* FIT201 = Sensor 201 measuring water flow rate
* LIT301 = Sensor 301 measuring water level at Tank 301

Please refer to [SWat Testbed description](https://minicps.readthedocs.io/en/latest/swat-tutorial.html?highlight=plc1#supply-and-storage-control) for details of Supply and Storage control in water treatment plant.

### B. Unseen Attack 2

* Method: Keep MV101 open and manipulate LIT101 to a constant rate of 0.7

* Result: tank T101 overflow

#### 1.Data Preparation

In [36]:
# Upload a file from local machine to the Google Colab environment
from google.colab import files
import io
import pandas as pd

uploaded = files.upload()

# Assuming you know the filename or if there's only one file uploaded
filename = next(iter(uploaded))
unseen_data2 = pd.read_csv(io.BytesIO(uploaded[filename]), parse_dates=['time'])
unseen_data2['time'] = unseen_data2['time'].dt.strftime('%Y-%m-%d %H:%M:%S')


Saving attack21_recorded.csv to attack21_recorded.csv


In [37]:
# Check the number of rows and columns in the dataframe
print("The number of rows and columns: ", unseen_data2.shape)

The number of rows and columns:  (71, 7)


In [38]:
# Print the first 5 rows of the dataframe
print("The first 5 rows of the dataset: \n", unseen_data2.head())

The first 5 rows of the dataset: 
                   time  FIT101  MV101  LIT101  P101  FIT201    LIT301
0  2023-07-31 14:49:12    2.55      1     0.7     1    2.45  0.507037
1  2023-07-31 14:49:13    2.55      1     0.7     1    2.45  0.524630
2  2023-07-31 14:49:14    2.55      1     0.7     1    2.45  0.542222
3  2023-07-31 14:49:15    2.55      1     0.7     1    2.45  0.559815
4  2023-07-31 14:49:16    2.55      1     0.7     1    2.45  0.577407


In [39]:
# Check the time span of the unseen data
unseen_data2['time'] = pd.to_datetime(unseen_data2['time'])
time_span = unseen_data2.time.max() - unseen_data2.time.min()
print("Time span of the dataset:", time_span)

Time span of the dataset: 0 days 00:01:10


In [40]:
# Plot a graph to see how data looks like
import plotly.express as px
fig = px.line(unseen_data2, x="time", y=['FIT101', 'MV101', 'LIT101', 'P101', 'FIT201', 'LIT301'], title='TRIST unseen data - attack 2 (Keep MV101 open and manipulate LIT101 to a constant rate of 0.7)', template = 'plotly_dark')
fig.show()

In [41]:
# Algorithms cannot directly consume date or timestamp data, thus we will extract the features from the timestamp
# and will drop the actual timestamp column before training models.
# Set the timestamp column as the index of the dataframe
unseen_data2.set_index('time', drop=True, inplace=True)

In [42]:
# Extract features from timestamp
unseen_data2['hours'] = [i.hour for i in unseen_data2.index]
unseen_data2['minutes'] = [i.minute for i in unseen_data2.index]
unseen_data2['seconds']= [i.second for i in unseen_data2.index]

print("\nThe first 5 rows of the dataset: \n", unseen_data2.head())

print("\nThe last 5 rows of the dataset: \n", unseen_data2.tail())


The first 5 rows of the dataset: 
                      FIT101  MV101  LIT101  P101  FIT201    LIT301  hours  \
time                                                                        
2023-07-31 14:49:12    2.55      1     0.7     1    2.45  0.507037     14   
2023-07-31 14:49:13    2.55      1     0.7     1    2.45  0.524630     14   
2023-07-31 14:49:14    2.55      1     0.7     1    2.45  0.542222     14   
2023-07-31 14:49:15    2.55      1     0.7     1    2.45  0.559815     14   
2023-07-31 14:49:16    2.55      1     0.7     1    2.45  0.577407     14   

                     minutes  seconds  
time                                   
2023-07-31 14:49:12       49       12  
2023-07-31 14:49:13       49       13  
2023-07-31 14:49:14       49       14  
2023-07-31 14:49:15       49       15  
2023-07-31 14:49:16       49       16  

The last 5 rows of the dataset: 
                      FIT101  MV101  LIT101  P101  FIT201    LIT301  hours  \
time                            

#### 2.Prediction

In [43]:
# Predict on test set
# The predict_model function returns Anomaly and Anomaly_Score label as a new column in the input dataframe.
# The predict_model is only useful when you want to obtain labels on unseen data (i.e. data that was not used during training the model).
iforest_pred2 = predict_model(iforest, data=unseen_data2)
iforest_pred2

,FIT101,MV101,LIT101,P101,FIT201,LIT301,hours,minutes,seconds,Anomaly,Anomaly_Score
time,,,,,,,,,,,
2023-07-31 14:49:12,2.55,1.0,0.7,1.0,2.45,0.507037,14.0,49.0,12.0,1,0.016453
2023-07-31 14:49:13,2.55,1.0,0.7,1.0,2.45,0.524630,14.0,49.0,13.0,1,0.014899
2023-07-31 14:49:14,2.55,1.0,0.7,1.0,2.45,0.542222,14.0,49.0,14.0,1,0.015237
2023-07-31 14:49:15,2.55,1.0,0.7,1.0,2.45,0.559815,14.0,49.0,15.0,1,0.014840
2023-07-31 14:49:16,2.55,1.0,0.7,1.0,2.45,0.577407,14.0,49.0,16.0,1,0.012466
...,...,...,...,...,...,...,...,...,...,...,...
2023-07-31 14:50:18,2.55,1.0,0.7,1.0,0.00,0.663519,14.0,50.0,18.0,1,0.024900
2023-07-31 14:50:19,2.55,1.0,0.7,1.0,0.00,0.635741,14.0,50.0,19.0,1,0.025185
2023-07-31 14:50:20,2.55,1.0,0.7,1.0,0.00,0.607963,14.0,50.0,20.0,1,0.023336


In [44]:
# Print the predicted anomalies
print("Anomalies: \n", iforest_pred2[iforest_pred2['Anomaly'] == 1])

Anomalies: 
                      FIT101  MV101  LIT101  P101  FIT201    LIT301  hours  \
time                                                                        
2023-07-31 14:49:12    2.55    1.0     0.7   1.0    2.45  0.507037   14.0   
2023-07-31 14:49:13    2.55    1.0     0.7   1.0    2.45  0.524630   14.0   
2023-07-31 14:49:14    2.55    1.0     0.7   1.0    2.45  0.542222   14.0   
2023-07-31 14:49:15    2.55    1.0     0.7   1.0    2.45  0.559815   14.0   
2023-07-31 14:49:16    2.55    1.0     0.7   1.0    2.45  0.577407   14.0   
2023-07-31 14:49:17    2.55    1.0     0.7   1.0    2.45  0.595000   14.0   
2023-07-31 14:49:18    2.55    1.0     0.7   1.0    2.45  0.612593   14.0   
2023-07-31 14:49:19    2.55    1.0     0.7   1.0    2.45  0.630185   14.0   
2023-07-31 14:49:20    2.55    1.0     0.7   1.0    2.45  0.644259   14.0   
2023-07-31 14:49:21    2.55    1.0     0.7   1.0    2.45  0.661852   14.0   
2023-07-31 14:49:22    2.55    1.0     0.7   1.0    2.45  0.679

In [45]:
# Count the number of anomalies
number_of_anomalies = iforest_pred2['Anomaly'].sum()

print("Number of anomalies detected in attack 2:", number_of_anomalies)

Number of anomalies detected in attack 2: 33


In [46]:
# Recall the number of rows and columns in the dataset
print("The number of rows and columns: ", unseen_data2.shape)

The number of rows and columns:  (71, 9)


The number of anomalies detected is 33, representing 46% of the 71 records in the datasets.

In [47]:
# Plot anomalies
import plotly.graph_objects as go

# Plot value on y-axis and date on x-axis
fig = px.line(iforest_pred2, x=iforest_pred2.index, y=['FIT101', 'MV101', 'LIT101', 'P101', 'FIT201', 'LIT301'], title='TRIST Anomaly Detection - Prediction on Unseen Attack 2 (Keep MV101 open and manipulate LIT101 to a constant rate of 0.7)', template = 'plotly_dark')

# Create list of outliers
outliers = iforest_pred2[iforest_pred2['Anomaly'] == 1].index

# obtain y value of anomalies to plot
y_values1 = [iforest_pred2.loc[i]['FIT101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values1, mode = 'markers',
                name = 'FIT101 Anomalies',
                marker=dict(color='yellow',size=10)))

# obtain y value of anomalies to plot
y_values2 = [iforest_pred2.loc[i]['MV101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values2, mode = 'markers',
                name = 'MV101 Anomalies',
                marker=dict(color='white',size=10)))

# obtain y value of anomalies to plot
y_values3 = [iforest_pred2.loc[i]['LIT101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values3, mode = 'markers',
                name = 'LIT101 Anomalies',
                marker=dict(color='pink',size=10)))

# obtain y value of anomalies to plot
y_values4 = [iforest_pred2.loc[i]['P101'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values4, mode = 'markers',
                name = 'P101 Anomalies',
                marker=dict(color='lightcyan',size=10)))

# obtain y value of anomalies to plot
y_values5 = [iforest_pred2.loc[i]['FIT201'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values5, mode = 'markers',
                name = 'FIT201 Anomalies',
                marker=dict(color='lightgoldenrodyellow',size=10)))

# obtain y value of anomalies to plot
y_values6 = [iforest_pred2.loc[i]['LIT301'] for i in outliers]

fig.add_trace(go.Scatter(x=outliers, y=y_values6, mode = 'markers',
                name = 'LIT301 Anomalies',
                marker=dict(color='lightyellow',size=10)))

fig.show()

Explanation for the above graph:
* FIT101 = Sensor 101 measuring water flow rate
* MV101 = Motorised valve 101
* LIT101 = Sensor 101 measuring water level at Tank 101
* P101 = Pump 101
* FIT201 = Sensor 201 measuring water flow rate
* LIT301 = Sensor 301 measuring water level at Tank 301

Please refer to [SWat Testbed description](https://minicps.readthedocs.io/en/latest/swat-tutorial.html?highlight=plc1#supply-and-storage-control) for details of Supply and Storage control in water treatment plant.